<p>
  <a href="https://colab.research.google.com/github/ezhilvendhan/ecommerce-demo-gds-vertex-ai/blob/main/similarity.ipynb" target="_blank">
    <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
  </a>
</p>

# Install Prerequisites
First off, you'll also need to install a few packages.

In [ ]:
%pip install --quiet --upgrade graphdatascience
%pip install --quiet google-cloud-storage
%pip install --quiet google.cloud.aiplatform

# Restart the Kernel
After you install the additional packages, you need to restart the notebook kernel so it can find the packages.  When you run this, you may get a notification that the kernel crashed.  You can disregard that.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

# Working with Neo4j
You'll need to enter the credentials from your Neo4j instance below.  You can get these by running the command ":server connect" in the Neo4j Browser.  The default DB_USER and DB_NAME are always neo4j.

In [9]:
# Edit these variables!
DB_URL = "neo4j+s://c1c3e9b6.databases.neo4j.io:7687"
DB_PASS = ""

# You can leave this default
DB_USER = 'neo4j'

In [10]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(DB_URL, auth=(DB_USER, DB_PASS), aura_ds=True)

# Data Import

## Create Constraints

In [56]:
result = gds.run_cypher(
  """
    CREATE CONSTRAINT childCategoryIdConstraint IF NOT EXISTS FOR (c:Category) REQUIRE c.id IS UNIQUE
  """
)
display(result)

""


In [57]:
result = gds.run_cypher(
  """
    CREATE CONSTRAINT attributeIdConstraint IF NOT EXISTS FOR (a:Attribute) REQUIRE a.id IS UNIQUE
  """
)
display(result)

""


In [58]:
result = gds.run_cypher(
  """
    CREATE CONSTRAINT productIdConstraint IF NOT EXISTS FOR (p:Product) REQUIRE p.id IS UNIQUE
  """
)
display(result)

""


In [ ]:
result = gds.run_cypher(
  """
    SHOW CONSTRAINTS YIELD id, name, type, entityType, labelsOrTypes, properties, ownedIndexId;
  """
)
display(result)

## Load Data

Create Nodes

In [60]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/main/data/import_node_category.csv' AS line WITH line
    MERGE (n:Category {id:line.id,name:line.name})
  """
)
display(result)

""


In [61]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/main/data/import_node_sku.csv' AS line WITH line
    MERGE (n:SKU {id:line.id,name:line.name,attributes:line.attributes,brand:line.brand,colour:line.colour,serial:line.serial})
  """
)
display(result)

""


In [62]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/main/data/import_node_keyword.csv' AS line WITH line
    MERGE (n:Keyword {id:line.id,keywords:line.keywords})
  """
)
display(result)

""


In [63]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/main/data/import_node_product.csv' AS line WITH line
    MERGE (n:Product {id:line.id,shop_id:line.shop_id,name:line.name})
  """
)
display(result)

""


In [64]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/main/data/import_node_attribute.csv' AS line WITH line
    MERGE (n:Attribute {id:line.id,type:line.type,value:line.value})
  """
)
display(result)

""


Create Relationships

In [65]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/main/data/import_relation_IS_CATEGORY.csv' AS line 
    WITH line
    MATCH (from:SKU {id:line.sku_id}), (to:Category {id:line.category_id})
    CREATE (from)-[:IS_CATEGORY]->(to)
  """
)
display(result)

""


In [66]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/main/data/import_relation_IS_SKU.csv' AS line 
    WITH line
    MATCH (from:Product {id:line.product_id}), (to:SKU {id:line.sku_id})
    CREATE (from)-[:IS_SKU]->(to)
  """
)
display(result)

""


In [67]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/main/data/import_relation_WITH_KEYWORD.csv' AS line 
    WITH line
    MATCH (from:SKU {id:line.sku_id}), (to:Keyword {id:line.keyword_id})
    CREATE (from)-[:WITH_KEYWORD]->(to)
  """
)
display(result)

""


In [68]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/main/data/import_relation_HOT_SALE.csv' AS line 
    WITH line
    MATCH (from:SKU {id:line.sku_id}), (to:Product {id:line.product_id})
    CREATE (from)-[:HOT_SALE]->(to)
  """
)
display(result)

""


In [69]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/main/data/import_relation_SUPPLEMENT_WITH.csv' AS line 
    WITH line
    MATCH (from:SKU {id:line.from_sku_id}), (to:SKU {id:line.to_sku_id})
    CREATE (from)-[:SUPPLEMENT_WITH]->(to)
  """
)
display(result)

""


In [70]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/main/data/import_relation_LOW_PRICE.csv' AS line 
    WITH line
    MATCH (from:SKU {id:line.sku_id}), (to:Product {id:line.product_id})
    CREATE (from)-[:LOW_PRICE]->(to)
  """
)
display(result)

""


## Graph Data Science

First we're going to create an in memory graph represtation of the data in Neo4j Graph Data Science (GDS).

In [71]:
result = gds.run_cypher(
  """
    CALL gds.graph.project('similarity-graph',
      ['Product', 'SKU'],
      '*'
    )
  """
)
display(result)

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
0,"{'Product': {'label': 'Product', 'properties':...","{'__ALL__': {'orientation': 'NATURAL', 'aggreg...",similarity-graph,51000,60559,75


Note, if you get an error saying the graph already exists, that's probably because you ran this code before. You can destroy it using this command:

In [55]:
result = gds.run_cypher(
  """
    CALL gds.graph.drop('similarity-graph')
  """
)
display(result)

,graphName,database,memoryUsage,sizeInBytes,nodeCount,relationshipCount,configuration,density,creationTime,modificationTime,schema
0,similarity-graph,neo4j,,-1,5100,56550,{'relationshipProjection': {'__ALL__': {'orien...,0.002175,2022-07-06T03:05:48.619649000+00:00,2022-07-06T03:22:26.023872000+00:00,"{'graphProperties': {}, 'relationships': {'__A..."


Now, let's list the details of the graph to make sure the projection was created as we want.

In [72]:
result = gds.run_cypher(
  """
    CALL gds.graph.list()
  """
)
display(result)

,degreeDistribution,graphName,database,memoryUsage,sizeInBytes,nodeCount,relationshipCount,configuration,density,creationTime,modificationTime,schema
0,"{'p99': 10, 'min': 1, 'max': 22, 'mean': 1.187...",similarity-graph,neo4j,7071 KiB,7241493,51000,60559,{'relationshipProjection': {'__ALL__': {'orien...,0.000023,2022-07-06T06:56:17.179008000+00:00,2022-07-06T06:56:17.253591000+00:00,"{'graphProperties': {}, 'relationships': {'__A..."


Nowe we can use NodeSimilarity Algorithm to find similar nodes. You can learn more about that [here](https://neo4j.com/docs/graph-data-science/current/algorithms/node-similarity/).

In [73]:
result = gds.run_cypher(
  """
  CALL gds.nodeSimilarity.mutate('similarity-graph',{
    mutateRelationshipType: 'SIMILAR',
    mutateProperty:'score'
  })
  """
)
display(result)

,preProcessingMillis,computeMillis,mutateMillis,postProcessingMillis,nodesCompared,relationshipsWritten,similarityDistribution,configuration
0,0,26434,108,-1,51000,505000,"{'p1': 1.0000073909759521, 'max': 1.0000073909...","{'topK': 10, 'similarityMetric': 'JACCARD', 'b..."


In [74]:
result = gds.run_cypher(
  """
  CALL gds.nodeSimilarity.write('similarity-graph', {
      writeRelationshipType: 'SIMILAR',
      writeProperty: 'score'
  })
  YIELD nodesCompared, relationshipsWritten
  """
)
display(result)

,nodesCompared,relationshipsWritten
0,51000,509600


In [75]:
result = gds.run_cypher(
  """
    MATCH (n:SKU) RETURN n
  """
)
display(result)

,n
0,"(colour, serial, name, attributes, id, brand)"
1,"(colour, serial, name, attributes, id, brand)"
2,"(colour, serial, name, attributes, id, brand)"
3,"(colour, serial, name, attributes, id, brand)"
4,"(colour, serial, name, attributes, id, brand)"
...,...
995,"(colour, serial, name, attributes, id, brand)"
996,"(colour, serial, name, attributes, id, brand)"
997,"(colour, serial, name, attributes, id, brand)"
998,"(colour, serial, name, attributes, id, brand)"


To demonstrate interoperability, let's fetch the data from Neo4J and run a Classification model from Vertex AI. 

In [76]:
import pandas as pd
df = pd.DataFrame([dict(record.items()) for record in result['n']])
df = df.drop(columns=['attributes'])
df['class'] = df.apply(lambda row: True if (len(row['brand']) < 15) else False,axis=1)
df

,colour,serial,name,id,brand,class
0,Aquamarine,81901,sku_name_0,0,Bolton Burns and Turner,False
1,BlanchedAlmond,46697,sku_name_1,1,Mann-Kelley,True
2,DarkSlateBlue,68373,sku_name_2,2,Alvarez-Davis,True
3,Aquamarine,70118,sku_name_3,3,Martinez and Sons,False
4,Chartreuse,99259,sku_name_4,4,Martinez and Sons,False
...,...,...,...,...,...,...
995,PapayaWhip,89164,sku_name_995,995,Greer Wilson and Davis,False
996,DarkSlateBlue,36019,sku_name_996,996,Johnson Inc,True
997,MediumSeaGreen,99259,sku_name_997,997,Mcdowell Inc,True
998,Wheat,89738,sku_name_998,998,Sanchez Ltd,True


Now that we have the data formatted properly, let's split it into a training and a testing set and write those to disk.

In [77]:

df.to_csv('raw.csv', index=False)
df

,colour,serial,name,id,brand,class
0,Aquamarine,81901,sku_name_0,0,Bolton Burns and Turner,False
1,BlanchedAlmond,46697,sku_name_1,1,Mann-Kelley,True
2,DarkSlateBlue,68373,sku_name_2,2,Alvarez-Davis,True
3,Aquamarine,70118,sku_name_3,3,Martinez and Sons,False
4,Chartreuse,99259,sku_name_4,4,Martinez and Sons,False
...,...,...,...,...,...,...
995,PapayaWhip,89164,sku_name_995,995,Greer Wilson and Davis,False
996,DarkSlateBlue,36019,sku_name_996,996,Johnson Inc,True
997,MediumSeaGreen,99259,sku_name_997,997,Mcdowell Inc,True
998,Wheat,89738,sku_name_998,998,Sanchez Ltd,True


# Authenticate your Google Cloud Account
Now let's write the file to Google Cloud Storage so we can use it in our model.  To do so, we must first authenticate.

Edit the variables below.  You can find the project ID in the Google Cloud Console.  The STORAGE_BUCKET is the name of a new bucket.  It must be globally unique.  It also needs to be all lower case.

In [78]:
# Edit this variable!
PROJECT_ID = 'neo4jbusinessdev'

# You can leave these defaults
STORAGE_BUCKET = PROJECT_ID + '-ev-form13'
REGION = 'us-central1'

In [79]:
import os
os.environ['GCLOUD_PROJECT'] = PROJECT_ID

In [80]:
try:
    from google.colab import auth as google_auth
    google_auth.authenticate_user()
except:
    pass

# Upload to Google Cloud Storage
Now we can upload our data sets to our bucket.

In [81]:
from google.cloud import storage
client = storage.Client()

Run the code below to create bucket, if needed. If the bucket exists, you get an error

In [82]:
bucket = client.bucket(STORAGE_BUCKET)
if(client.get_bucket(bucket) is None):
  bucket.location=REGION
  client.create_bucket(bucket)

In [83]:
filename='raw.csv'
upload_path = os.path.join('similarity', filename)
blob = bucket.blob(upload_path)
blob.upload_from_filename(filename)

# Train a Model on GCP
We'll use the original features to train an AutoML model.

In [84]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

dataset = aiplatform.TabularDataset.create(
    display_name="similarity-raw",
    gcs_source=os.path.join("gs://", STORAGE_BUCKET, 'similarity', 'raw.csv'),
)
dataset.wait()

print(f'\tDataset: "{dataset.display_name}"')
print(f'\tname: "{dataset.resource_name}"')

Creating TabularDataset
Create TabularDataset backing LRO: projects/803648085855/locations/us-central1/datasets/3469456164889034752/operations/2067024204577898496
TabularDataset created. Resource name: projects/803648085855/locations/us-central1/datasets/3469456164889034752
To use this TabularDataset in another session:
ds = aiplatform.TabularDataset('projects/803648085855/locations/us-central1/datasets/3469456164889034752')
	Dataset: "similarity-raw"
	name: "projects/803648085855/locations/us-central1/datasets/3469456164889034752"


In [85]:
job = aiplatform.AutoMLTabularTrainingJob(
    display_name='similarity-raw',
    optimization_prediction_type='classification'
)

In [86]:
model = job.run(
    dataset=dataset, 
    target_column="class", 
    training_fraction_split=0.8, 
    validation_fraction_split=0.1, 
    test_fraction_split=0.1, 
    model_display_name="similarity-raw", 
    disable_early_stopping=False, 
    budget_milli_node_hours=1000, 
)

No column transformations provided, so now retrieving columns from dataset in order to set default column transformations.
The column transformation of type 'auto' was set for the following columns: ['name', 'id', 'colour', 'brand', 'serial'].
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/7226168813505478656?project=803648085855
AutoMLTabularTrainingJob projects/803648085855/locations/us-central1/trainingPipelines/7226168813505478656 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLTabularTrainingJob projects/803648085855/locations/us-central1/trainingPipelines/7226168813505478656 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLTabularTrainingJob projects/803648085855/locations/us-central1/trainingPipelines/7226168813505478656 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLTabularTrainingJob projects/803648085855/locations/us-central1/trainingPipelines/7226168813505478656 current state:
PipelineState.PIPELINE_ST

1000 milli node hours, or one node hour, is the minimum budget that Vertex AI allows.  However, Vertex AI isn't respecting that budget currently.  This job will probably run for two and a half hours.  

We're going to move on while that runs.  You can check on the job later in the [Google Cloud Console](https://console.cloud.google.com/) to see the results.  There's a link to the specific job in the output of the cell above.